### Notebook to calculate Pearson coefficient from real data using both time and temperature amplitude ratio, to see whether 1D hypothesis is actually relevant or not.

In [ ]:
#We get the data for point 46 (representing the highest k value, ie the most advective case)

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from pyheatmy import *
from pyheatmy.time_series_multiperiodic import *
import os as os

